Web应用程序的核心是让任何用户都能够注册账户并能够使用它，不管用户身处何方。在本章中，将创建一些表单，让用户能够添加主题和条目，以及编辑既有的条目。学习Django如何防范对基于表单的网页发起的常见攻击。

然后，将实现一个用户身份验证系统。创建一个注册页面，供用户创建账户，并让有些页面只能供已登录的用户访问。接下来，修改一些视图函数，使得用户只能看到自己的数据。学习如何确保用户数据的安全。

## 19.1　让用户能够输入数据
建立用于创建用户账户的身份验证系统之前，先添加几个页面，让用户能够输入数据。让用户能够添加新主题、添加新条目以及编辑既有条目。

当前，只有超级用户能够通过管理网站输入数据。不让用户与管理网站交互，因此使用Django的表单创建工具来创建让用户能够输入数据的页面。

### 19.1.1　添加新主题
首先来让用户能够添加新主题。创建基于表单的页面的方法几乎与前面创建网页一样：定义一个URL，编写一个视图函数并编写一个模板。一个主要差别是，需要导入包含表单的模块forms.py。

#### 19.1.1.1 用于添加主题的表单

让用户输入并提交信息的页面都是`表单`。用户输入信息时，需要进行验证，确认提供的信息是正确的数据类型，且不是恶意的信息。然后，再对这些信息进行处理，并将其保存到数据库的合适地方。这些工作很多都是由Django自动完成的。

在Django中，创建表单的最简单方式是使用ModelForm，根据我们在第18章定义的模型中的信息自动创建表单。创建一个名为forms.py的文件，将其存储到models.py所在的目录中，并在其中编写一个表单。

```forms.py
from django import forms
from .models import Topic

class TopicForm(forms.ModelForm):
    """表单类"""
    class Meta:
        model = Topic
        fields = ['text']
        labels = {'text': ''}
```

首先导入了模块forms以及要使用的模型Topic 。定义了一个名为TopicForm 的类，继承了forms.ModelForm。

最简单的ModelForm版本只包含一个内嵌的Meta类，告诉Django根据哪个模型创建表单，以及在表单中包含哪些字段。模型Topic创建一个表单，该表单只包含字段text。`labels = {'text': ''}`让Django不要为字段text 生成标签。

#### 19.1.1.2 URL模式new_topic
这个新网页的URL应简短而具有描述性，因此当用户要添加新主题时，将切换到http://IP:8000/new_topic/ 。下面是网页new_topic的URL模式，将其添加到learning_logs/urls.py。

```urls.py
urlpatterns = [
    # 主页
    path(r'', views.index, name='index'),
    
    # 显示所有的主题
    url(r'^topics/$', views.topics, name='topics'),
    
    # 显示主题下的条目
    url(r'^topics/(?P<topic_id>\d+)/$', views.topic, name='topic'),
    
    # 用于添加新主题
    url(r'^newtopic/$', views.new_topic,name='new_topic'),
    ]
```

#### 19.1.1.3 视图函数new_topic()
函数new_topic() 需要处理两种情形：刚进入new_topic网页（显示一个空表单）；对提交的表单数据进行处理，并将用户重定向到网页topics。

```views.py
from django.http import HttpResponseRedirect
from django.urls import reverse

from .forms import TopicForm

def new_topic(request):
    """添加新主题"""
    if request.method != 'POST':
        # 未提交数据：创建一个新表单
        form = TopicForm()
    else:
        # POST提交的数据,对数据进行处理
        form = TopicForm(request.POST)
        if form.is_valid():
            form.save()
            return HttpResponseRedirect(reverse('learning_logs:topics'))
    
    context = {'form:': form}
    return render(request, 'learning_logs/new_topic.html', context)
```

导入了HttpResponseRedirect类，用户提交主题后将使用这个类将用户重定向到网页topics 。函数reverse()根据指定的URL模型确定URL，Django将在页面被请求时生成URL。还导入了刚才创建的表单TopicForm 。

#### 19.1.1.4 GET请求和POST请求

创建Web应用程序时，将用到的两种主要请求类型是GET请求和POST请求。对于只是从服务器读取数据的页面，使用GET请求；在用户需要通过表单提交信息时，通常使用POST请求。处理所有表单时，都将指定使用POST方法。还有一些其他类型的请求，但这个项目没有使用。

函数new_topic()将请求对象作为参数。用户初次请求该网页时，其浏览器将发送GET请求；用户填写并提交表单时，其浏览器将发送POST请求。根据请求的类型，可以确定用户请求的是空表单（GET请求）还是要求对填写好的表单进行处理（POST请求）。

`if request.method != 'POST':`处的测试确定请求方法是GET还是POST。如果请求方法不是POST，请求就可能是GET，因此返回一个空表单（即便请求是其他类型的，返回一个空表单也不会有任何问题）。创建一个TopicForm 实例，将其存储在变量form中，再通过上下文字典将这个表单发送给模板`context = {'form:': form}`。由于实例化TopicForm 时没有指定任何实参，Django将创建一个可供用户填写的空表单。

如果请求方法为POST，将执行else代码块，对提交的表单数据进行处理。使用用户输入的数据（存储在request.POST中）创建一个TopicForm 实例，这样对象form将包含用户提交的信息。

要将提交的信息保存到数据库，必须先通过检查确定它们是有效的。函数is_valid() 核实用户填写了所有必不可少的字段（表单字段默认都是必不可少的），且输入的数据与要求的字段类型一致（例如，字段text 少于200个字符）。这种自动验证避免了大量的工作。如果所有字段都有效，就可调用save()，将表单中的数据写入数据库。保存数据后，就可离开这个页面了。使用reverse() 获取页面topics的URL，并将其传递给HttpResponseRedirect()，后者将用户的浏览器重定向到页面topics。在页面topics 中，用户将在主题列表中看到刚输入的主题。

#### 19.1.1.5 模板new_topic

下面来创建新模板new_topic.html，用于显示刚创建的表单。

```html
{% extends "learning_logs/base.html" %}

{% block content %}
    <p>Add a new topic:</p>
    <form action="{% url 'learning_logs:new_topic' %}" method='post'>
        {% csrf_token %}
        {{ form.as_p }}
        <button name="submit">add topic</button>
    </form>

{% endblock content %}
```

这个模板继承了base.html，因此其基本结构与项目“学习笔记”的其他页面相同。首先定义了一个HTML表单。实参action告诉服务器将提交的表单数据发送到哪里，这里将它发回给视图函数new_topic() 。实参method让浏览器以POST请求的方式提交数据。

Django使用模板标签{% csrf_token %}来防止攻击者利用表单来获得对服务器未经授权的访问（这种攻击被称为跨站请求伪造 ）。接着显示表单，只需包含模板变量{{ form.as_p }} ，就可让Django自动创建显示表单所需的全部字段。修饰符as_p让Django以段落格式渲染所有表单元素，这是一种整洁地显示表单的简单方式。

Django不会为表单创建提交按钮，因此定义了一个这样的按钮。

#### 19.1.1.6 链接到页面new_topic
在页面topics 中添加一个到页面new_topic 的链接

```topics.html
{% extends "learning_logs/base.html" %}

{% block content %}

    <p>Topics</p>
    
    <ul>
        {% for topic in topics %}
        <li>
            <a href="{% url 'learning_logs:topic' topic.id %}">{{ topic }}</a>
        </li>
        {% empty %}
        <li>No topics have been added yet.</li>
        {% endfor %}
    </ul>

    <a href="{% url 'learning_logs:new_topic' %}">Add a new topic:</a>

{% endblock content %}
```

这个链接放在了既有主题列表的后面。

### 19.1.2　添加新条目

现在用户可以添加新主题了，但还要添加新条目。再次定义URL，编写视图函数和模板，并链接到添加新条目的网页。在此之前，需要在forms.py中再添加一个类。

#### 19.1.2.1 用于添加新条目的表单
创建一个与模型Entry 相关联的表单，但这个表单的定制程度比TopicForm 要高些

```forms.py
from .models import Topic, Entry

class EntryForm(forms.ModelForm):
    """条目类"""
    class Meta:
        model = Entry
        fields = ['text']
        labels = {'text': ''}
        widgets = {'text': forms.Textarea(attrs={'cols': 80})}
```

首先修改了import语句，使其除导入Topic外，还导入Entry。新类EntryForm继承了forms.ModelForm，包含的Meta类指出了表单基于的模型以及要在表单中包含哪些字段。这里也给字段'text'指定了一个空标签。

接着定义了属性widgets 。小部件（`widget`）是一个HTML表单元素，如单行文本框、多行文本区域或下拉列表。通过设置属性widgets ，可覆盖Django选择的默认小部件。通过让Django使用forms.Textarea，定制了字段'text'的输入小部件，将文本区域的宽度设置为80列，而不是默认的40列。

#### 19.1.2.2 URL模式new_entry
在用于添加新条目的页面的URL模式中，需要包含实参topic_id，因为条目必须与特定的主题相关联。该URL模式如下，将它添加到了learning_logs/urls.py中。

```urls.py
urlpatterns = [
    # 主页
    path(r'', views.index, name='index'),
    
    # 显示所有的主题
    url(r'^topics/$', views.topics, name='topics'),
    
    # 显示主题下的条目
    url(r'^topics/(?P<topic_id>\d+)/$', views.topic, name='topic'),
    
    # 用于添加新主题
    url(r'^newtopic/$', views.new_topic,name='new_topic'),
    
    # 用于添加新条目
    url(r'^newentry/(?P<topic_id>\d+)$', views.new_entry,name='new_entry'),
    ]
```

这个URL模式与形式为http://localhost:8000/new_entry/id / 的URL匹配，其中id 是一个与主题ID匹配的数字。代码(`?P<topic_id>\d+`) 捕获一个数字值，并将其存储在变量topic_id中。请求的URL与这个模式匹配时，Django将请求和主题ID发送给函数new_entry() 。

#### 19.1.2.3 视图函数new_entry()

```views.py
from .forms import TopicForm, EntryForm

def new_entry(request, topic_id):
    """在特定的主题中添加新条目"""
    topic = Topic.objects.get(id=topic_id)

    """添加新条目"""
    if request.method != 'POST':
        # 未提交数据,创建一个空表单
        form = EntryForm()
    else:
        # POST提交的数据,对数据进行处理
        form = EntryForm(data=request.POST)
        if form.is_valid():
            new_entry = form.save(commit=False)
            new_entry.topic = topic
            new_entry.save()
            return HttpResponseRedirect(reverse('learning_logs:topic', args=[topic_id]))

    context = {'form': form}
    return render(request, 'learning_logs/new_entry.html', context)
```

修改import 语句，在其中包含了刚创建的EntryForm 。new_entry()的定义包含形参topic_id，用于存储从URL中获得的值。渲染页面以及处理表单数据时，都需要知道针对的是哪个主题，因此使用topic_id来获得正确的主题。

检查请求方法是POST还是GET。如果是GET请求，将执行if 代码块：创建一个空的EntryForm 实例。如果请求方法为POST，就对数据进行处理：创建一个EntryForm实例，使用request对象中的POST数据来填充它；再检查表单是否有效，如果有效，就设置条目对象的属性topic ，再将条目对象保存到数据库。

调用save() 时，传递了实参commit=False，让Django创建一个新的条目对象，并将其存储到new_entry中，但不将它保存到数据库中。将new_entry的属性topic设置为在这个函数开头从数据库中获取的主题，然后调用save() ，且不指定任何实参。这将把条目保存到数据库，并将其与正确的主题相关联。

最后将用户重定向到显示相关主题的页面。调用reverse()时，需要提供两个实参：要根据它来生成URL的URL模式的名称；列表args，其中包含要包含在URL中的所有实参。在这里，列表args只有一个元素——topic_id 。接下来，调用HttpResponseRedirect()将用户重定向到显示新增条目所属主题的页面，用户将在该页面的条目列表中看到新添加的条目。

#### 19.1.2.4 模板new_entry
模板new_entry 类似于模板new_topic：

```new_entry.html
{% extends "learning_logs/base.html" %}

{% block content %}

    <p><a href="{% url 'learning_logs:topic' topic.id %}">{{ topic }}</a></p>

    <p>Add a new entry:</p>
    <form action="{% url 'learning_logs:new_entry' topic.id %}" method='post'>
        {% csrf_token %}
        {{ form.as_p }}
        <button name="submit">add entry</button>
    </form>

{% endblock content %}
```

在页面顶端显示了主题，让用户知道他是在哪个主题中添加条目；该主题名也是一个链接，可用于返回到该主题的主页面。

表单的实参action包含URL中的topic_id 值，让视图函数能够将新条目关联到正确的主题。除此之外，这个模板与模板new_topic.html完全相同。

#### 19.1.2.5 链接到页面new_entry
在显示特定主题的页面中添加到页面new_entry 的链接：

```topic.html
{% extends 'learning_logs/base.html' %}

{% block content %}
topic.html:
    <p>Topic: {{ topic }}</p>
    
    <p>Entries:</p>
    <p>
        <a href="{% url 'learning_logs:new_entry' topic.id %}">add new entry</a>
    </p>
    <ul>
    {% for entry in entries %}
        <li>
            <p>{{ entry.date_added|date:'M d, Y H:i' }}</p>
            <p>{{ entry.text|linebreaks }}</p>
        </li>
    {% empty %}
        <li>
            There are no entries for this topic yet.
        </li>
    {% endfor %}
    </ul>

{% endblock content %}
```

在显示条目前添加链接，执行的最常见的操作是添加新条目。现在用户可以添加新主题，还可以在每个主题中添加任意数量的条目。

### 19.1.3　编辑条目
下面来创建一个页面，让用户能够编辑既有的条目。

#### 19.1.3.1 URL模式edit_entry
这个页面的URL需要传递要编辑的条目的ID。修改后的learning_logs/urls.py如下：

```urls.py
    # 用于编辑条目
    url(r'^editentry/(?P<entry_id>\d+)/$', views.edit_entry, name='edit_entry'),
```

在URL中传递的ID存储在形参entry_id中。这个URL模式将预期匹配的请求发送给视图函数edit_entry() 。

#### 19.1.3.2 视图函数edit_entry()
页面edit_entry收到GET请求时，edit_entry()返回一个表单，让用户能够对条目进行编辑。该页面收到POST请求（条目文本经过修订）时，它将修改后的文本保存到数据库中：

```views.py
from .models import Topic, Entry

def edit_entry(request, entry_id):
    """编辑特定的条目"""
    entry = Entry.objects.get(id=entry_id)
    topic = entry.topic

    if request.method != 'POST':
        # 初次请求，使用当前条目填充表单
        form = EntryForm(instance=entry)
    else:
        # POST提交的数据，对数据进行处理
        form = EntryForm(instance=entry, data=request.POST)
        if form.is_valid():
            form.save()
            return HttpResponseRedirect(reverse('learning_logs:topic', args=[topic.id]))

    context = {'entry': entry, 'topic': topic, 'form': form}
    return render(request, 'learning_logs/edit_entry.html', context)
```

首先导入模型Entry。获取用户要修改的条目对象，以及与该条目相关联的主题。在请求方法为GET时将执行的if代码块中，使用实参`instance=entry`创建一个EntryForm实例。这个实参让Django创建一个表单，并使用既有条目对象中的信息填充它。用户将看到既有的数据，并能够编辑它们。

处理POST请求时，传递实参instance=entry和data=request.POST，让Django根据既有条目对象创建一个表单实例，并根据request.POST中的相关数据对其进行修改。然后，检查表单是否有效，如果有效，就调用save() ，且不指定任何实参。接下来，重定向到显示条目所属主题的页面，用户将在其中看到其编辑的条目的新版本。

#### 19.1.2.3 模板edit_entry

```edit_entry.html
{% extends "learning_logs/base.html" %}

{% block content %}
    <p>
        <a href="{% url 'learning_logs:topic' topic.id %}">{{ topic }}</a>
    </p>

    <p>Edit entry:</p>
    
    <form action="{% url 'learning_logs:edit_entry' entry.id %}" method='post'>
        {% csrf_token %}
        {{ form.as_p }}
        <button name="submit">save changes</button>
    </form>

{% endblock content %}
```

实参action将表单发回给函数edit_entry() 进行处理。在标签{% url %} 中，将条目ID作为一个实参，让视图对象能够修改正确的条目对象。将提交按钮命名为save changes，以提醒用户：单击该按钮将保存所做的编辑，而不是创建一个新条目。

#### 19.1.2.4 链接到页面edit_entry
在显示特定主题的页面中，需要给每个条目添加到页面edit_entry 的链接：

```topic.html
{% extends 'learning_logs/base.html' %}

{% block content %}
topic.html:
    <p>Topic: {{ topic }}</p>
    
    <p>Entries:</p>
    <p>
        <a href="{% url 'learning_logs:new_entry' topic.id %}">add new entry</a>
    </p>
    <ul>
    {% for entry in entries %}
        <li>
            <p>{{ entry.date_added|date:'M d, Y H:i' }}</p>
            <p>{{ entry.text|linebreaks }}</p>
            <p>
                <a href="{% url 'learning_logs:edit_entry' entry.id %}">edit entry</a>
            </p>
        </li>
    {% empty %}
        <li>
            There are no entries for this topic yet.
        </li>
    {% endfor %}
    </ul>

{% endblock content %}
```

将编辑链接放在每个条目的日期和文本后面。在循环中，使用模板标签{% url %} 根据URL模式edit_entry和当前条目的ID属性（entry.id）来确定URL。链接文本为"edit entry"，出现在页面中每个条目的后面。

### **练习：**

19-1 博客 ：新建一个Django项目，将其命名为Blog。在这个项目中，创建一个名为blogs的应用程序，并在其中创建一个名为BlogPost 的模型。这个模型应包含title 、text 和date_added 等字段。为这个项目创建一个超级用户，并使用管理网站创建几个简短的帖子。创建一个主页，在其中按时间顺序显示所有的帖子。

创建两个表单，其中一个用于发布新帖子，另一个用于编辑既有的帖子。

尝试填写这些表单，确认它们能够正确地工作。

## 19.2　创建用户账户

将建立一个用户注册和身份验证系统，让用户能够注册账户，进而登录和注销。创建一个新的应用程序，其中包含与处理用户账户相关的所有功能。将对模型Topic 稍做修改，让每个主题都归属于特定用户。

### 19.2.1　应用程序users
首先使用命令startapp 来创建一个名为users 的应用程序：

```sh
python3 manage.py startapp users
```

#### 19.2.1.1 将应用程序users 添加到settings.py中

在settings.py中，将这个新的应用程序添加到INSTALLED_APPS中

```settings.py
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    # My APP
    'learning_logs',
    'users',
]
```

#### 19.2.1.2 包含应用程序users 的URL
修改项目根目录中的urls.py，使其包含应用程序users定义的URL。

```urls.py
urlpatterns = [
    path('admin/', admin.site.urls),
    path(r'', include(('learning_logs.urls','learning_logs') , namespace='learning_logs')),
    url(r'^users/', include('users.urls', namespace='users')),
    ]
```

添加了一行代码，以包含应用程序users中的文件urls.py。这行代码与任何以单词users打头的URL都匹配。创建了命名空间'users'，以便将应用程序learning_logs的URL同应用程序users的URL区分开来。

### 19.2.2　登录页面
首先实现登录页面的功能。为此，使用Django提供的默认登录视图，因此URL模式会稍有不同。在目录learning_log/users/中，新建一个名为urls.py的文件，并在其中添加如下代码：

```urls.py
from django.conf.urls import url
from django.contrib.auth.views import LoginView

from . import views

urlpatterns =[
    # 登录页面
    url(r'^login/$', LoginView.as_view(template_name='users/login.html'), name="login"),
]

```

首先导入了默认视图LoginView。登录页面的URL模式与URL http://IP:8000/users/login/ 匹配。这个URL中的单词users让Django在users/urls.py中查找，而单词login让它将请求发送给Django默认视图login。鉴于没有编写自己的视图函数，传递了一个template_name参数，告诉Django去哪里查找编写的模板。这个模板包含在应用程序users而不是learning_logs中。

#### 19.2.2.1 模板login.html
用户请求登录页面时，Django将使用其默认视图login ，但依然需要为这个页面提供模板。为此，在目录learning_log/users/中，创建一个名为templates的目录，并在其中创建一个名为users的目录。以下是模板login.html，将其存储到目录learning_log/users/templates/users/中：

```login.html
{% extends "learning_logs/base.html" %}

{% block content %}
    {% if form.errors %}
        <p>Your username and password didn't match. Please try again.</p>
    {% endif %}
    
    <form method="post" action="{% url 'users:login' %}">
        {% csrf_token %}
        {{ form.as_p }}
    <button name="submit">log in</button>
    <input type="hidden" name="next" value="{% url 'learning_logs:index' %}" />
    </form>

{% endblock content %}
```

这个模板继承了base.html，旨在确保登录页面的外观与网站的其他页面相同。

> 一个应用程序中的模板可继承另一个应用程序中的模板。

如果表单的errors 属性被设置，就显示一条错误消息，指出输入的用户名—密码对与数据库中存储的任何用户名—密码对都不匹配。

让登录视图处理表单，因此将实参action设置为登录页面的URL。登录视图将一个表单发送给模板，在模板中，显示这个表单并添加一个提交按钮。接着包含一个隐藏的表单元素——'next' ，其中的实参value告诉Django在用户成功登录后将其重定向到什么地方——在这里是主页。

#### 19.2.2.2 链接到登录页面
在base.html中添加到登录页面的链接，让所有页面都包含它。用户已登录时，不显示这个链接，因此将它嵌套在一个{% if %} 标签中：

```base.html
<p>
    base.html: 
    <a href="{% url 'learning_logs:index' %}">Learning Log</a>
    <a href="{% url 'learning_logs:topics' %}">Topics</a>
    {% if user.is_authenticated %}
        Hello, {{ user.username }}.
    {% else %}
        <a href="{% url 'users:login' %}">login</a>
    {% endif %}
</p>
    
{% block content %}{% endblock content %}
```

在Django身份验证系统中，每个模板都可使用变量user ，这个变量有一个is_authenticated属性：如果用户已登录，该属性将为True，否则为False 。这能够向已通过身份验证的用户显示一条消息，而向未通过身份验证的用户显示另一条消息。

在这里，向已登录的用户显示一条问候语。对于已通过身份验证的用户，还设置了属性username ，使用这个属性来个性化问候语，让用户知道他已登录。对于还未通过身份验证的用户，再显示一个到登录页面的链接。

#### 19.2.2.3 使用登录页面
前面建立了一个用户账户，登录一下，看看登录页面是否管用。请访问http://IP:8000/admin/ ，如果是以管理员的身份登录的，请在页眉上找到注销链接并单击它。

注销后，访问http://IP:8000/users/login/ ，将看到登录页面。输入设置的用户名和密码，将进入页面index。在这个主页的页眉中，显示了一条个性化问候语，其中包含用户名。

### 19.2.3　注销
提供一个让用户注销的途径。不创建用于注销的页面，而让用户只需单击一个链接就能注销并返回到主页。为此，将注销链接定义一个URL模式，编写一个视图函数，并在base.html中添加一个注销链接。

#### 19.2.3.1 注销URL
为注销定义了URL模式，该模式与URL http://IP:8000/users/logout/ 匹配。修改后的users/urls.py如下

```urls.py
"""为应用程序users定义URL模式"""

from django.conf.urls import url
from django.contrib.auth.views import LoginView
from django.contrib.auth import login

from . import views

app_name = 'users'

urlpatterns =[
    # 登录页面
    url(r'^login/$', LoginView.as_view(template_name='users/login.html'), name="login"),
    
    # 注销页面
    url(r'^logout/$', views.logout_view, name="logout"),
]
```

这个URL模式将请求发送给函数logout_view() 。这样给这个函数命名，旨在将其与将在其中调用的函数logout()区分开来
> 请确保修改的是users/urls.py，而不是learning_log/urls.py。

#### 19.2.3.2. 视图函数logout_view()
函数logout_view()：导入Django函数logout()，并调用它，再重定向到主页。打开users/views.py:

```views.py
from django.shortcuts import render

from django.http import HttpResponseRedirect
from django.urls import reverse
from django.contrib.auth import logout

# Create your views here.
def logout_view(request):
    """注销用户"""
    logout(request)    
    return HttpResponseRedirect(reverse('learning_logs:index'))
```

从django.contrib.auth中导入了函数logout()。调用函数logout() ，将request对象作为实参。然后，重定向到主页。

#### 19.2.3.3 链接到注销视图
加一个注销链接。在base.html中添加这种链接，让每个页面都包含它；将它放在标签`{% if user.is_authenticated %}`中，使得仅当用户登录后才能看到。

```base.html
<p>
    base.html: 
    <a href="{% url 'learning_logs:index' %}">Learning Log</a>
    <a href="{% url 'learning_logs:topics' %}">Topics</a>
    {% if user.is_authenticated %}
        Hello, {{ user.username }}.
        <a href="{% url 'users:logout' %}">logout</a>
    {% else %}
        <a href="{% url 'users:login' %}">login</a>
    {% endif %}
</p>
    
{% block content %}{% endblock content %}
```

### 19.2.4　注册页面
创建一个让新用户能够注册的页面。使用Django提供的表单UserCreationForm ，但编写自己的视图函数和模板。

#### 19.2.4.1 注册页面的URL模式
下面的代码定义了注册页面的URL模式。
这个模式与URL http://IP:8000/users/register/ 匹配，并将请求发送给函数register()。 

```urls.py
"""为应用程序users定义URL模式"""

from django.conf.urls import url
from django.contrib.auth.views import LoginView
from django.contrib.auth import login

from . import views

app_name = 'users'

urlpatterns =[
    # 登录页面
    url(r'^login/$', LoginView.as_view(template_name='users/login.html'), name="login"),
    
    # 注销页面
    url(r'^logout/$', views.logout_view, name='logout'),
    
    # 注册页面
    url(r'^register/$', views.register, name='register')
]
```

#### 19.2.4.2 视图函数register()
注册页面首次被请求时，视图函数register()需要显示一个空的注册表单，并在用户提交填写好的注册表单时对其进行处理。如果注册成功，这个函数让用户自动登录。users/views.py中添加如下代码：

```views.py
from django.shortcuts import render

from django.http import HttpResponseRedirect
from django.urls import reverse
from django.contrib.auth import logout, login, authenticate
from django.contrib.auth.forms import UserCreationForm

def register(request):
    """注册用户"""
    if request.method != 'POST':
        # 显示空的注册表单
        form = UserCreationForm()
    else:
        # 处理已经填写好的表单
        form = UserCreationForm(data=request.POST)

        if form.is_valid():
            new_user = form.save()
            # 用户自动登录，然后重定向到主页
            authenticated_user = authenticate(username=new_user.username,
            password=request.POST('password1'))
            login(request, authenticated_user)
            return HttpResponseRedirect(reverse('learning_logs:index'))

    context = {'form':form}
    return render(request, 'users/register.html', context)
```

首先导入了函数render() ，然后导入函数login() 和authenticate()，以便在用户正确地填写了注册信息时让其自动登录。导入默认表单UserCreationForm。在函数register()中，检查要响应的是否是POST请求。如果不是，就创建一个UserCreationForm实例，且不给它提供任何初始数据。

如果响应的是POST请求，就根据提交的数据创建一个UserCreationForm实例，并检查这些数据是否有效：用户名未包含非法字符，输入的两个密码相同，以及用户没有试图做恶意的事情。

如果提交的数据有效，调用表单的方法save()，将用户名和密码的散列值保存到数据库中。方法save()返回新创建的用户对象，将其存储在new_user中。

保存用户的信息后，用户自动登录。这包含两个步骤：首先，调用authenticate()，并将实参new_user.username和密码传递给它。用户注册时，被要求输入密码两次；由于表单是有效的，输入的这两个密码是相同的，因此可以使用其中任何一个。从表单的POST数据中获取与键'password1'相关联的值。如果用户名和密码无误，方法authenticate()将返回一个通过了身份验证的用户对象，将其存储在authenticated_user中。接下来，调用函数login()，并将对象request和authenticated_user传递给它，这将为新用户创建有效的会话。最后，将用户重定向到主页，其页眉中显示了一条个性化的问候语，让用户知道注册成功了。

#### 19.2.4.3 注册模板
注册页面的模板与登录页面的模板类似，将其保存到login.html所在的目录中：

```register.html
{% extends "learning_logs/base.html" %}

{% block content %}

  <form method="post" action="{% url 'users:register' %}">
    {% csrf_token %}
    {{ form.as_p }}

    <button name="submit">register</button>
    <input type="hidden" name="next" value="{% url 'learning_logs:index' %}" />
  </form>

{% endblock content %}
```

这里使用方法as_p，让Django在表单中正确地显示所有的字段，包括错误消息——如果用户没有正确地填写表单。

#### 19.2.4.4 链接到注册页面
接下来，添加这样的代码，即在用户没有登录时显示到注册页面的链接：

```base.html
<p>
    base.html: 
    <a href="{% url 'learning_logs:index' %}">Learning Log</a>
    <a href="{% url 'learning_logs:topics' %}">Topics</a>
    {% if user.is_authenticated %}
        Hello, {{ user.username }}.
        <a href="{% url 'users:logout' %}">Logout</a>
    {% else %}
        <a href="{% url 'users:register' %}">Register</a> -
        <a href="{% url 'users:login' %}">Login</a>
    {% endif %}
</p>
    
{% block content %}{% endblock content %}
```

现在，已登录的用户看到的是个性化的问候语和注销链接，而未登录的用户看到的是注册链接和登录链接。

> 这里的注册系统允许用户创建任意数量的账户。有些系统要求用户确认其身份：发送一封确认邮件，用户回复后其账户才生效。通过这样做，系统生成的垃圾账户将比这里使用的简单系统少。

### **练习：**
19-2 博客账户 ：在你为完成练习19-1而开发的项目Blog中，添加一个用户身份验证和注册系统。让已登录的用户在屏幕上看到其用户名，并让未注册的用户看到一个到注册页面的链接。

## 19.3　让用户拥有自己的数据
用户应该能够输入其专有的数据，因此创建一个系统，确定各项数据所属的用户，再限制对页面的访问，让用户只能使用自己的数据。

在本节中，将修改模型Topic，让每个主题都归属于特定用户。这也将影响条目，因为每个条目都属于特定的主题。先来限制对一些页面的访问。

### 19.3.1　使用@login_required 限制访问
Django提供了装饰器`@login_required`，能够轻松地实现这样的目标：对于某些页面，只允许已登录的用户访问它们。`装饰器`（decorator）是放在函数定义前面的指令，Python在函数运行前，根据它来修改函数代码的行为。

#### 19.3.1.1 限制对topics 页面的访问
每个主题都归特定用户所有，因此应只允许已登录的用户请求topics页面。为此，在learning_logs/views.py中添加如下代码：

```views.py
from django.contrib.auth.decorators import login_required

@login_required
def topics(request):
    """显示所有的主题"""
    topics = Topic.objects.order_by('date_added')
    context = {'topics': topics}
    return render(request, 'learning_logs/topics.html', context)
```

首先导入了函数login_required()。将login_required()作为装饰器用于视图函数topics()——在它前面加上符号@和login_required，让Python在运行topics()的代码前先运行login_required()的代码。

login_required()的代码检查用户是否已登录，仅当用户已登录时，Django才运行topics()的代码。如果用户未登录，就重定向到登录页面。

为实现这种重定向，修改settings.py，让Django知道到哪里去查找登录页面。在settings.py末尾添加如下代码：

```settings.py
# My Settings
LOGIN_URL = '/users/login/'
```

现在，如果未登录的用户请求装饰器`@login_required`的保护页面，Django将重定向到settings.py中的LOGIN_URL指定的URL。

要测试这个设置，可注销并进入主页。然后，单击链接Topics，这将重定向到登录页面。接下来，使用你的账户登录，并再次单击主页中的Topics链接，你将看到topics页面。

#### 19.3.1.2 全面限制对项目“学习笔记”的访问
在项目“学习笔记”中，将不限制对主页、注册页面和注销页面的访问，并限制对其他所有页面的访问。

在下面的learning_logs/views.py中，对除index()外的每个视图都应用了装饰器@login_required。

### 19.3.2　将数据关联到用户
现在，需要将数据关联到提交它们的用户。只需将最高层的数据关联到用户，这样更低层的数据将自动关联到用户。

下面来修改模型Topic，在其中添加一个关联到用户的外键。这样做后，必须对数据库进行迁移。最后，必须对有些视图进行修改，使其只显示与当前登录的用户相关联的数据。

#### 19.3.2.1 修改模型Topic
修改models.py，导入了django.contrib.auth中的模型User ，然后在Topic中添加了字段owner，建立到模型User的外键关系。

```models.py
from django.contrib.auth.models import User

class Topic(models.Model):
    """用户学习的主题"""
    text = models.CharField(max_length=200)
    date_added = models.DateTimeField(auto_now_add=True)
    # 增加User的外键
    owner = models.ForeignKey(User, on_delete=models.CASCADE)

    def __str__(self):
        """返回模型的字符串表示"""
        return self.text
```

#### 19.3.2.2 确定当前有哪些用户
迁移数据库时，Django将对数据库进行修改，使其能够存储主题和用户之间的关联。为执行迁移，Django需要知道该将各个既有主题关联到哪个用户。最简单的办法是，将既有主题都关联到同一个用户，如超级用户。为此，需要知道该用户的ID。

下面来查看已创建的所有用户的ID。为此，启动一个Django shell会话，并执行如下命令：

```sh
$ python3 manage.py shell
Python 3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
(InteractiveConsole)
>>> from django.contrib.auth.models import User
>>> for user in User.objects.all():
...     print(user.username, user.id)
... 
ll_admin 1
ll_user 2
```

首先在shell会话中导入了模型User 。然后，查看到目前为止都创建了哪些用户。输出中列出了三个用户：ll_admin、ll_user。

接着遍历用户列表，并打印每位用户的用户名和ID。Django询问要将既有主题关联到哪个用户时，将指定其中的一个ID值。

#### 19.3.2.3 迁移数据库

知道用户ID后，就可以迁移数据库了。

```sh
$ python3 manage.py makemigrations learning_logs
You are trying to add a non-nullable field 'owner' to topic without a default; we can't do that (the database needs something to populate existing rows).
Please select a fix:
 1) Provide a one-off default now (will be set on all existing rows with a null value for this column)
 2) Quit, and let me add a default in models.py
Select an option: 1
Please enter the default value now, as valid Python
The datetime and django.utils.timezone modules are available, so you can do e.g. timezone.now
Type 'exit' to exit this prompt
>>> 1
Migrations for 'learning_logs':
  learning_logs/migrations/0003_topic_owner.py
    - Add field owner to topic
```

首先执行了命令makemigrations。在输出中，Django指出试图给既有模型Topic添加一个必不可少（不可为空）的字段，而该字段没有默认值。接着Django提供了两种选择：要么现在提供默认值，要么退出并在models.py中添加默认值。选择了第一个选项，因此Django提示输入默认值。

为将所有既有主题都关联到管理用户ll_admin，输入用户ID值1。并非必须使用超级用户，而可使用已创建的任何用户的ID。接下来，Django使用这个值来迁移数据库，并生成了迁移文件0003_topic_owner.py，在模型Topic中添加字段owner 。

现在可以执行迁移了。为此，在活动的虚拟环境中执行下面的命令：

```sh
$ python3 manage.py migrate                     
Operations to perform:
  Apply all migrations: admin, auth, contenttypes, learning_logs, sessions
Running migrations:
  Applying learning_logs.0003_topic_owner... OK
```

Django应用新的迁移，结果一切顺利。

为验证迁移符合预期，可在shell会话中像下面这样做：

```sh
$ python3 manage.py shell                       
Python 3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
(InteractiveConsole)
>>> from learning_logs.models import Topic
>>> for topic in Topic.objects.all():
...     print(topic, topic.owner)
... 
Chess ll_admin
Rock Climbing ll_admin
Jogging ll_admin
```

从learning_logs.models 中导入Topic，再遍历所有的既有主题，并打印每个主题及其所属的用户。正如看到的，现在每个主题都属于用户ll_admin。

> 可以`重置`数据库而不是迁移它，但如果这样做，既有的数据都将丢失。一种不错的做法是，学习如何在迁移数据库的同时确保用户数据的完整性。如果确实想要一个全新的数据库，可执行命令`python manage.py flush`，这将重建数据库的结构。如果这样做，就必须重新创建超级用户，且原来的所有数据都将丢失。

### 19.3.3　只允许用户访问自己的主题
当前，不管以哪个用户的身份登录，都能够看到所有的主题。现在来调整，只向用户显示属于自己的主题。

在views.py中，对函数topics()做如下修改：

```views.py
@login_required
def topics(request):
    """显示所有的主题"""
    #topics = Topic.objects.order_by('date_added')
    # 仅显示用户所拥有的主题
    topics = Topic.objects.filter(owner=request.user).order_by('date_added')
    context = {'topics': topics}
    return render(request, 'learning_logs/topics.html', context)
```

用户登录后，request 对象将有一个user 属性，这个属性存储了有关该用户的信息。代码Topic.objects.filter(owner=request.user) 让Django只从数据库中获取owner属性为当前用户的Topic对象。由于没有修改主题的显示方式，因此无需对页面topics的模板做任何修改。

要查看结果，以所有既有主题关联到的用户的身份登录，并访问topics页面，将看到所有的主题。然后，注销并以另一个用户的身份登录，topics页面将不会列出任何主题。

### 19.3.4　保护用户的主题
还没有限制对显示单个主题的页面的访问，因此任何已登录的用户都可输入类似于http://IP:8000/topics/1/ 的URL，来访问显示相应主题的页面。

为修复这种问题，在视图函数topic()获取请求的条目前执行检查。

```views.py
from django.http import HttpResponseRedirect, Http404

@login_required
def topic(request, topic_id):
    """显示单个主题及其所有的条目"""
    topic = Topic.objects.get(id=topic_id)
    # 确认请求的主题属于当前的用户
    if topic.owner != request.user:
        raise Http404
    entries = topic.entry_set.order_by('-date_added')
    context = {'topic': topic, 'entries': entries}
    return render(request, 'learning_logs/topic.html', context)
```

服务器上没有请求的资源时，标准的做法是返回404响应。在这里，导入了异常Http404，并在用户请求它不能查看的主题时引发这个异常。收到主题请求后，在渲染网页前检查该主题是否属于当前登录的用户。如果请求的主题不归当前用户所有，就引发Http404异常，让Django返回一个404错误页面。

### 19.3.5　保护页面edit_entry

页面edit_entry 的URL为http://IP:8000/edit_entry/entry_id/ ，其中entry_id是一个数字。下面来保护这个页面，禁止用户通过输入类似于前面的URL来访问其他用户的条目：

```views.py
@login_required
def edit_entry(request, entry_id):
    """编辑特定的条目"""
    entry = Entry.objects.get(id=entry_id)
    topic = entry.topic

    if topic.owner != request.user:
        raise Http404

    if request.method != 'POST':
        # 初次请求，使用当前条目填充表单
        form = EntryForm(instance=entry)
    else:
        # POST提交的数据，对数据进行处理
        form = EntryForm(instance=entry, data=request.POST)
        if form.is_valid():
            form.save()
            return HttpResponseRedirect(reverse('learning_logs:topic', args=[topic.id]))
```

### 19.3.6　将新主题关联到当前用户
当前，用于添加新主题的页面存在问题，因此没有将新主题关联到特定用户。如果尝试添加新主题，将看到错误消息IntegrityError，指出learning_logs_topic.user_id不能为NULL 。Django的意思是说，创建新主题时，必须指定其owner字段的值。

可以通过request对象获悉当前用户，因此存在一个修复这种问题的简单方案。

```views.py
@login_required
def new_topic(request):
    """添加新主题"""
    if request.method != 'POST':
        # 未提交数据：创建一个新表单
        form = TopicForm()
    else:
        # POST提交的数据,对数据进行处理
        form = TopicForm(request.POST)
        if form.is_valid():
            new_topic = form.save(commit=False)
            new_topic.owner = request.user
            new_topic.save()
            return HttpResponseRedirect(reverse('learning_logs:topics'))
    
    context = {'form': form}
    return render(request, 'learning_logs/new_topic.html', context)
```

首先调用form.save()，并传递实参commit=False，这是因为先修改新主题，再将其保存到数据库中。接下来，将新主题的owner属性设置为当前用户。最后，对刚定义的主题实例调用save()。现在主题包含所有必不可少的数据，将被成功地保存。

现在，这个项目允许任何用户注册，而每个用户想添加多少新主题都可以。每个用户都只能访问自己的数据，无论是查看数据、输入新数据还是修改旧数据时都如此。

### **练习：**
19-3 重构 ：在views.py中，在两个地方核实主题关联到的用户为当前登录的用户。请将执行这种检查的代码放在一个名为check_topic_owner() 的函数中，并在恰当的地方调用这个函数。

19-4 保护页面new_entry ：一个用户可在另一个用户的学习笔记中添加条目，方法是输入这样的URL，即其中包含输入另一个用户的主题的ID。为防范这种攻击，请在保存新条目前，核实它所属的主题归当前用户所有。

19-5 受保护的博客 ：在你创建的项目Blog中，确保每篇博文都与特定用户相关联。确保任何用户都可访问所有的博文，但只有已登录的用户能够发表博文以及编辑既有博文。在让用户能够编辑其博文的视图中，在处理表单前确认用户编辑的是他自己发表的博文。